In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/MyDrive/learning/nus/cs5260/project'
data_path = root_path + '/data'
cleaned_data_path = root_path + '/data/cleaned'

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
emails = pd.read_csv(data_path + '/ling.csv')

emails.rename(columns={'label': 'spam'}, inplace=True)

# Fill empty string for empty / NA value in Subject and Message columns
emails['subject'].fillna('', inplace=True)
emails['message'].fillna('', inplace=True)

# Create a new column called text by concatenating Subject and Message
emails['text'] = emails['subject'] + ' ' + emails['message']
emails['original_text'] = emails['text']
emails.drop(columns=['subject', 'message'], inplace=True)

# Count by spam status
emails['spam'].value_counts()

spam
0    2412
1     481
Name: count, dtype: int64

In [ ]:
print('Before removing duplicate emails:', emails.shape)

emails.drop_duplicates(inplace=True)
print('After removing duplicate emails:', emails.shape)

emails

In [ ]:
import ast

def parse_and_concatenate(value):
    try:
        # Attempt to parse the string as a list
        parsed_value = ast.literal_eval(value)
        if isinstance(parsed_value, list):
            # Join the list elements into a single string if parsing is successful
            return ' '.join(str(item) for item in parsed_value)
    except (ValueError, SyntaxError):
        # Return the original value if parsing fails or if it's not a list
        return value

    # Return the original value if it's not a list
    return value

# Apply the function to the DataFrame column
emails['text'] = emails['text'].apply(parse_and_concatenate)

In [ ]:
from bs4 import BeautifulSoup

def extract_text(html):
    soup = BeautifulSoup(html, "lxml")
    return soup.get_text()

emails['text'] = emails['text'].apply(extract_text)

<ipython-input-5-5df6bfd0c925>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "lxml")


In [ ]:
emails['spam'].value_counts()

spam
0    2408
1     468
Name: count, dtype: int64

In [ ]:
emails[emails['spam'] == 1].sample(10)

,spam,text,original_text
616,1,some think so ! ! ! ! ! ! ! ! - - adults only ...,some think so ! ! ! ! ! ! ! ! - - adults only ...
720,1,secret hardcore sex site . . . click here now ...,secret hardcore sex site . . . click here now ...
1075,1,* we purchase uncollected judgments - - we pur...,* we purchase uncollected judgments - - we pur...
189,1,hey : ) click here to recieve your free 1 mont...,hey : ) click here to recieve your free 1 mont...
190,1,the ultimate free xxx experience ! ! this is a...,the ultimate free xxx experience ! ! this is a...
2555,1,"data to : = 20 date : fri , 06 feb 1998 22 : 3...","data to : = 20 date : fri , 06 feb 1998 22 : ..."
1039,1,re : hello * * * * * * warning ! this message ...,re : hello * * * * * * warning ! this message ...
647,1,"> > > holiday in florida ! < < < ` , , ` , , `...","> > > holiday in florida ! < < < ` , , ` , , `..."
2474,1,""" free "" $ 400 bulk mail software "" explodes y...",""" free "" $ 400 bulk mail software "" explodes y..."
923,1,"dental - optical plan hello , we work with a g...","dental - optical plan hello , we work with a g..."


In [ ]:
emails[emails['spam'] == 0].sample(10)

,spam,text,original_text
412,0,"' de proverbio ' news hi , i have some news re...","' de proverbio ' news hi , i have some news re..."
1321,0,meeting notice fourth mid-continental workshop...,meeting notice fourth mid-continental workshop...
760,0,new books on human cognitive processing john b...,new books on human cognitive processing john b...
1204,0,books on latin linguistics john benjamins publ...,books on latin linguistics john benjamins publ...
2191,0,internet - accessible linguistic data-sources ...,internet - accessible linguistic data-sources ...
1823,0,"discourse , anaphora and reference resolution ...","discourse , anaphora and reference resolution ..."
77,0,sapir - whorf sapir-whorf hypothesis i know li...,sapir - whorf sapir-whorf hypothesis i know li...
2593,0,the 13th annual international conference on pr...,the 13th annual international conference on pr...
537,0,sum : singapore english content - length : 426...,sum : singapore english content - length : 426...
2443,0,slavic linguistics fasl8 8th annual workshop o...,slavic linguistics fasl8 8th annual workshop o...


In [ ]:
emails['text'] = emails['text'].apply(lambda x: x.replace("\\n", " ").replace("\\r", " ").replace("\\t", " "))
emails['text'] = emails['text'].str.replace(r'\s+', ' ', regex=True).str.replace(
    '[^a-zA-Z0-9 \!\?\.,;\:\-\—\(\)\[\]\{\}\'\"%\$&\*\/=@#\\+]',
    '',
    regex=True
)

emails

,spam,text,original_text
0,0,job posting - apple-iss research center conten...,job posting - apple-iss research center conten...
1,0,"lang classification grimes , joseph e . and ba...","lang classification grimes , joseph e . and b..."
2,0,query : letter frequencies for text identifica...,query : letter frequencies for text identifica...
3,0,risk a colleague and i are researching the dif...,risk a colleague and i are researching the dif...
4,0,request book information earlier this morning ...,request book information earlier this morning ...
...,...,...,...
2888,1,love your profile - ysuolvpv hello thanks for ...,love your profile - ysuolvpv hello thanks for ...
2889,1,you have been asked to join kiddin the list ow...,you have been asked to join kiddin the list ow...
2890,0,anglicization of composers ' names judging fro...,anglicization of composers ' names judging fro...
2891,0,"re : 6 . 797 , comparative method : n - ary co...","re : 6 . 797 , comparative method : n - ary co..."


In [ ]:
emails['original_text'] = emails['text']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize(text):
    return ' '.join(tokenizer.tokenize(text))
emails['text'] = emails['text'].apply(tokenize)

emails.head()

,spam,text,original_text
0,0,job posting - apple - iss research center cont...,job posting - apple-iss research center conten...
1,0,"lang classification grimes , joseph e . and ba...","lang classification grimes , joseph e . and b..."
2,0,query : letter frequencies for text identifica...,query : letter frequencies for text identifica...
3,0,risk a colleague and i are researching the dif...,risk a colleague and i are researching the dif...
4,0,request book information earlier this morning ...,request book information earlier this morning ...


In [ ]:
print(emails.shape)

# Strip whitespace from the text column
emails['text'] = emails['text'].str.strip()

# Remove emails with empty text
emails = emails[emails['text'].str.len() > 50]

print(emails.shape)

(2876, 3)
(2874, 3)


In [ ]:
# Find duplicate emails
duplicate_emails = emails[emails.duplicated(subset='text', keep=False)]
duplicate_emails.sort_values(by='text')

,spam,text,original_text


In [ ]:
emails.loc[647]['text']

'holiday in florida !     , ,  , ,  , ,  florida / bahamas / cancun family holiday ! ! ! !  , ,  , ,  , ,  you have been selected to enter for a limited time only ! family getaway for 2 adults 3 children or 4 adults ! offer now available world wide for a limited time only ! if selected you could enjoy : 1 . 4 days 3 nights in magical orlando florida , home of disney world where you will enjoy the hospitality of either the comfort suites or holiday inn . have a blast with a free family pass to watermania water park in orlando . you will also receive the orlando " magic card " entitling you to hundreds of dollars worth of discounts in the orlando area ! ! ! ! ! ! 2 . then enjoy 3 days 3 nighs in sunny cocoa beach florida , living it up at the hilton overlooking the atlantic ocean ! ! ! put the kids to bed early . . . . . . and enjoy 2 complimentary adult casino cruise tickets ! ! ! ! ! 3 . a complimentary alamo rental car with unlimited mileage for your stay in florida ( must carry a mjo

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import re

# Ensure you have the necessary NLTK datasets downloaded
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

english_words = set(words.words())
lemmatizer = nltk.WordNetLemmatizer()

def count_non_english_words(text):
    # Tokenize the text into words
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    tokens = word_tokenize(text)
    # Lematize tokens using lematizer
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    # Count total words and non-English words
    total_words = len(tokens)
    non_english_words = sum(1 for word in tokens if word not in english_words)

    # Return the count of non-English words and total words
    return non_english_words, total_words

# Apply the function to the 'text' column and calculate the percentage of non-English words
emails['non_english_info'] = emails['original_text'].apply(count_non_english_words)
emails['percentage_non_english'] = emails['non_english_info'].apply(lambda x: x[0] / x[1] if x[1] > 0 else 0)

# Filter emails containing more than 30% non-English words
filtered_emails = emails[emails['percentage_non_english'] > 0.60].sort_values(by='percentage_non_english', ascending=False)

filtered_emails.shape

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-14-2cce9f71d457>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emails['non_english_info'] = emails['original_text'].apply(count_non_english_words)
<ipython-input-14-2cce9f71d457>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

(34, 5)

In [ ]:
# Remove filtered_emails from email dataframe
print(emails.shape)
emails = emails[~emails.index.isin(filtered_emails.index)]
print(emails.shape)

(2874, 5)
(2840, 5)


In [ ]:
# Count unique words in text column
unique_words = set()
for text in emails['text']:
    unique_words.update(text.split())

print(len(unique_words))

63070


In [ ]:
emails['spam'].value_counts()

0    2376
1     466
Name: spam, dtype: int64

In [ ]:
emails[['text', 'original_text', 'spam']].to_csv(cleaned_data_path + '/ling.csv', index=False)